In [1]:
import requests
import time 
import json
import os
from bs4 import BeautifulSoup
import pandas as pd

In [42]:
def get_date_data(country, store):
    response = requests.get(f'https://ana-slo.com/ホールデータ/{country}/{store}-データ一覧/')
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('div' , id= 'table')

    #找到所有有 a 的日期
    a = table.find_all('a')
    a_list = []
    for date in a:
        a_list.append(date.text.strip()[:10].replace('/', '-'))

    return a_list

In [43]:
a_list = get_date_data('北海道','ロイヤル網走店')

In [156]:
#the main function
from threading import Thread
class ana_slo_crawler:

    def __init__(self, country, store):
        response = requests.get(f'https://ana-slo.com/ホールデータ/{country}/{store}-データ一覧/')
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('div' , id= 'table')

        #找到所有有 a 的日期
        a = table.find_all('a')
        a_list = []
        for date in a:
            a_list.append(date.text.strip()[:10].replace('/', '-'))

        self.a_list = a_list
        self.store = store 
        self.columns_slot = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', '合成確率', 'BB確率', 'RB確率','DATE']

    def get_slot_data_14(self):  

        def _get_slot_data(date , store):
            nonlocal data_list

            response = requests.get(f'https://ana-slo.com/{date}-{store}-data/')
            soup = BeautifulSoup(response.text, 'html.parser')

            div = soup.find_all('div', id='all_data_block')
            tr = div[0].find_all('tr')
            columns_slot = tr[0].text.strip().split('\n')

            for td in tr[1:len(tr)]:
                data = td.text.strip().split('\n')
                if len(data)==len(columns_slot):
                    data.append(date)
                    data_list.append(data)

        data_list = []
        ths = [None] * len(self.a_list[:14])
        for i in range(len(self.a_list[:14])):
            ths[i] = Thread(target=_get_slot_data, args=(self.a_list[i],self.store ), daemon=True)
            ths[i].start()

        for i in range(len(self.a_list[:14])):
            ths[i].join()   
            
        return data_list

In [165]:
data = ana_slo_crawler('北海道','ロイヤル網走店')
data.a_list
data.store

'ロイヤル網走店'

In [166]:
data.a_list[:14]

['2024-08-28',
 '2024-08-26',
 '2024-08-18',
 '2024-08-16',
 '2024-08-08',
 '2024-08-06',
 '2024-07-28',
 '2024-07-26',
 '2024-07-18',
 '2024-07-16',
 '2024-07-08',
 '2024-07-06',
 '2024-06-28',
 '2024-06-26']

In [167]:
df = pd.DataFrame(data.get_slot_data_14(), columns=data.columns_slot)

In [175]:
df['DATE'].unique()

array(['2024-08-18', '2024-06-28', '2024-08-28', '2024-07-26',
       '2024-08-16', '2024-07-06', '2024-07-18', '2024-08-08',
       '2024-08-26', '2024-07-28', '2024-06-26', '2024-07-08',
       '2024-07-16', '2024-08-06'], dtype=object)

In [100]:
response = requests.get(f'https://ana-slo.com/2024-08-28-ロイヤル網走店-data/')
soup = BeautifulSoup(response.text, 'html.parser')

div = soup.find_all('div', id='all_data_block')
tr = div[0].find_all('tr')
columns_slot = tr[0].text.strip().split('\n')

In [123]:
columns_slot

['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', '合成確率', 'BB確率', 'RB確率']

In [104]:
for td in tr[1:len(tr)]:
    data = td.text.strip().split('\n')
    data.append('2024-08-28')
    print(data)

['アイムジャグラーEX-TP', '482', '4,291', '-350', '14', '14', '1/153.2', '1/306.5', '1/306.5', '2024-08-28']
['アイムジャグラーEX-TP', '484', '1,711', '-1,022', '3', '2', '1/342.2', '1/570.3', '1/855.5', '2024-08-28']
['アイムジャグラーEX-TP', '486', '4,478', '+2,068', '20', '26', '1/97.3', '1/223.9', '1/172.2', '2024-08-28']
['アイムジャグラーEX-TP', '488', '4,143', '-450', '11', '15', '1/159.3', '1/376.6', '1/276.2', '2024-08-28']
['アイムジャグラーEX-TP', '490', '7,133', '+35', '25', '25', '1/142.7', '1/285.3', '1/285.3', '2024-08-28']
['アイムジャグラーEX-TP', '492', '4,097', '-622', '15', '8', '1/178.1', '1/273.1', '1/512.1', '2024-08-28']
['アイムジャグラーEX-TP', '494', '7,806', '+1,735', '31', '35', '1/118.3', '1/251.8', '1/223.0', '2024-08-28']
['アイムジャグラーEX-TP', '496', '6,154', '+78', '23', '20', '1/143.1', '1/267.6', '1/307.7', '2024-08-28']
['アイムジャグラーEX-TP', '498', '4,185', '-86', '15', '11', '1/161.0', '1/279.0', '1/380.5', '2024-08-28']
['アイムジャグラーEX-TP', '500', '8,106', '+1,939', '32', '37', '1/117.5', '1/253.3', '1/219.1', '20

In [59]:
a_list

['2024-08-28',
 '2024-08-26',
 '2024-08-18',
 '2024-08-16',
 '2024-08-08',
 '2024-08-06',
 '2024-07-28',
 '2024-07-26',
 '2024-07-18',
 '2024-07-16',
 '2024-07-08',
 '2024-07-06',
 '2024-06-28',
 '2024-06-26',
 '2024-06-18',
 '2024-06-16',
 '2024-06-08',
 '2024-06-06',
 '2024-05-28',
 '2024-05-26',
 '2024-05-18',
 '2024-05-16',
 '2024-05-08',
 '2024-05-06',
 '2024-04-28',
 '2024-04-26',
 '2024-04-18',
 '2024-04-16',
 '2024-04-08',
 '2024-04-06',
 '2024-03-28',
 '2024-03-26',
 '2024-03-24',
 '2024-03-21',
 '2024-03-18',
 '2024-03-16',
 '2024-03-08',
 '2024-03-06',
 '2024-02-28',
 '2024-02-26',
 '2024-02-18',
 '2024-02-16',
 '2024-02-08',
 '2024-02-06',
 '2024-01-28',
 '2024-01-26',
 '2024-01-18',
 '2024-01-16',
 '2024-01-08',
 '2024-01-06',
 '2023-12-28',
 '2023-12-26',
 '2023-12-18',
 '2023-12-16',
 '2023-12-08',
 '2023-12-06',
 '2023-11-28',
 '2023-11-26',
 '2023-11-18',
 '2023-11-16',
 '2023-11-08',
 '2023-11-06',
 '2023-10-28',
 '2023-10-26',
 '2023-10-18',
 '2023-10-16',
 '2023-10-

In [58]:
ths

NameError: name 'ths' is not defined

In [71]:
ths

NameError: name 'ths' is not defined

In [121]:
data = get_slot_data('ロイヤル網走店')


In [122]:
len(data)

2240

In [128]:
columns_slot.append('date')
columns_slot

['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', '合成確率', 'BB確率', 'RB確率', 'date']

In [129]:
df = pd.DataFrame(data , columns=columns_slot)
df

,機種名,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date
0,アイムジャグラーEX-TP,482,"4,291",-350,14,14,1/153.2,1/306.5,1/306.5,2024-08-28
1,アイムジャグラーEX-TP,484,"1,711","-1,022",3,2,1/342.2,1/570.3,1/855.5,2024-08-28
2,アイムジャグラーEX-TP,486,"4,478","+2,068",20,26,1/97.3,1/223.9,1/172.2,2024-08-28
3,アイムジャグラーEX-TP,488,"4,143",-450,11,15,1/159.3,1/376.6,1/276.2,2024-08-28
4,アイムジャグラーEX-TP,490,"7,133",+35,25,25,1/142.7,1/285.3,1/285.3,2024-08-28
...,...,...,...,...,...,...,...,...,...,...
2235,ソードアート・オンライン,471,0,0,0,0,1/0.0,1/0.0,1/0.0,2024-06-26
2236,ひぐらしのなく頃に 業,476,"1,606","+1,350",5,5,1/160.6,1/321.2,1/321.2,2024-06-26
2237,バイオハザード:ヴェンデッタ,518,132,-122,1,1,1/66.0,1/132.0,1/132.0,2024-06-26
2238,この素晴らしい世界に祝福を！,525,366,-29,8,2,1/36.6,1/45.8,1/183.0,2024-06-26


In [95]:
len(data)

2240

In [62]:
def _get_slot_data(date , store):
    response = requests.get(f'https://ana-slo.com/{date}-{store}-data/')
    soup = BeautifulSoup(response.text, 'html.parser')

    div = soup.find_all('div', id='all_data_block')
    tr = div[0].find_all('tr')
    columns_slot = tr[0].text.strip().split('\n')

    data_list = []
    for td in tr[1:len(tr)]:
        data = td.text.strip().split('\n')
        if len(data)==len(columns_slot):
            data_list.append(data)

    df = pd.DataFrame(data_list, columns=columns_slot)
    df['date'] = pd.to_datetime(date)

    df_all = pd.DataFrame(columns=columns_slot)
    df_all = pd.concat([df_all, df], axis=0, ignore_index=True)
    return df_all

In [67]:
for date in a_list[:14]:
    df = _get_slot_data(date ,'ロイヤル網走店' )
    if date == a_list[0]:
        df_all = df
    else:
        df_all = pd.concat([df_all, df], axis=0)
        

In [68]:
df_all

,機種名,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date
0,アイムジャグラーEX-TP,482,"4,291",-350,14,14,1/153.2,1/306.5,1/306.5,2024-08-28
1,アイムジャグラーEX-TP,484,"1,711","-1,022",3,2,1/342.2,1/570.3,1/855.5,2024-08-28
2,アイムジャグラーEX-TP,486,"4,478","+2,068",20,26,1/97.3,1/223.9,1/172.2,2024-08-28
3,アイムジャグラーEX-TP,488,"4,143",-450,11,15,1/159.3,1/376.6,1/276.2,2024-08-28
4,アイムジャグラーEX-TP,490,"7,133",+35,25,25,1/142.7,1/285.3,1/285.3,2024-08-28
...,...,...,...,...,...,...,...,...,...,...
155,ソードアート・オンライン,471,0,0,0,0,1/0.0,1/0.0,1/0.0,2024-06-26
156,ひぐらしのなく頃に 業,476,"1,606","+1,350",5,5,1/160.6,1/321.2,1/321.2,2024-06-26
157,バイオハザード:ヴェンデッタ,518,132,-122,1,1,1/66.0,1/132.0,1/132.0,2024-06-26
158,この素晴らしい世界に祝福を！,525,366,-29,8,2,1/36.6,1/45.8,1/183.0,2024-06-26


In [63]:
df_all = _get_slot_data('2024-08-28' ,'ロイヤル網走店' )
df_all 

,機種名,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date
0,アイムジャグラーEX-TP,482,"4,291",-350,14,14,1/153.2,1/306.5,1/306.5,2024-08-28
1,アイムジャグラーEX-TP,484,"1,711","-1,022",3,2,1/342.2,1/570.3,1/855.5,2024-08-28
2,アイムジャグラーEX-TP,486,"4,478","+2,068",20,26,1/97.3,1/223.9,1/172.2,2024-08-28
3,アイムジャグラーEX-TP,488,"4,143",-450,11,15,1/159.3,1/376.6,1/276.2,2024-08-28
4,アイムジャグラーEX-TP,490,"7,133",+35,25,25,1/142.7,1/285.3,1/285.3,2024-08-28
...,...,...,...,...,...,...,...,...,...,...
155,主役は銭形4,471,"2,683",+739,20,8,1/95.8,1/134.2,1/335.4,2024-08-28
156,ひぐらしのなく頃に 業,476,975,-350,2,2,1/243.8,1/487.5,1/487.5,2024-08-28
157,防空少女ラブキューレ2～極限の共鳴～,477,"1,572",+421,16,19,1/44.9,1/98.2,1/82.7,2024-08-28
158,この素晴らしい世界に祝福を！,525,"2,019","-1,793",19,7,1/77.7,1/106.3,1/288.4,2024-08-28


In [45]:
df_all = pd.DataFrame()
df_all
    

""


In [46]:
df = pd.read_csv('slot_data_マルハン石和店_20240514.csv')

In [39]:
df.head()

,機種名,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date
0,マイジャグラーV,682,"6,753","+2,500",31,30,1/110.7,1/217.8,1/225.1,2024-05-14
1,マイジャグラーV,683,"1,232",-700,2,3,1/246.4,1/616.0,1/410.7,2024-05-14
2,マイジャグラーV,684,"4,168",-500,15,9,1/173.7,1/277.9,1/463.1,2024-05-14
3,マイジャグラーV,685,"4,140",+600,18,14,1/129.4,1/230.0,1/295.7,2024-05-14
4,マイジャグラーV,686,"2,038",-700,5,6,1/185.3,1/407.6,1/339.7,2024-05-14


In [47]:
df_all

""


In [48]:
df

,機種名,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date
0,マイジャグラーV,682,"6,753","+2,500",31,30,1/110.7,1/217.8,1/225.1,2024-05-14
1,マイジャグラーV,683,"1,232",-700,2,3,1/246.4,1/616.0,1/410.7,2024-05-14
2,マイジャグラーV,684,"4,168",-500,15,9,1/173.7,1/277.9,1/463.1,2024-05-14
3,マイジャグラーV,685,"4,140",+600,18,14,1/129.4,1/230.0,1/295.7,2024-05-14
4,マイジャグラーV,686,"2,038",-700,5,6,1/185.3,1/407.6,1/339.7,2024-05-14
...,...,...,...,...,...,...,...,...,...,...
89037,アラジンAクラシック,670,"1,182",-500,11,2,1/90.9,1/107.5,1/591.0,2023-04-15
89038,頭文字D,671,"2,818","+1,200",20,24,1/64.0,1/140.9,1/117.4,2023-04-15
89039,バイオハザード7 レジデント イービル,672,"3,016",+100,45,4,1/61.6,1/67.0,1/754.0,2023-04-15
89040,モンキーターンⅣ,673,"2,289","-2,400",4,3,1/327.0,1/572.2,1/763.0,2023-04-15


In [49]:
df_all = pd.concat([df_all, df], axis=0 , ignore_index=True)

In [50]:
df_all

,機種名,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date
0,マイジャグラーV,682,"6,753","+2,500",31,30,1/110.7,1/217.8,1/225.1,2024-05-14
1,マイジャグラーV,683,"1,232",-700,2,3,1/246.4,1/616.0,1/410.7,2024-05-14
2,マイジャグラーV,684,"4,168",-500,15,9,1/173.7,1/277.9,1/463.1,2024-05-14
3,マイジャグラーV,685,"4,140",+600,18,14,1/129.4,1/230.0,1/295.7,2024-05-14
4,マイジャグラーV,686,"2,038",-700,5,6,1/185.3,1/407.6,1/339.7,2024-05-14
...,...,...,...,...,...,...,...,...,...,...
89037,アラジンAクラシック,670,"1,182",-500,11,2,1/90.9,1/107.5,1/591.0,2023-04-15
89038,頭文字D,671,"2,818","+1,200",20,24,1/64.0,1/140.9,1/117.4,2023-04-15
89039,バイオハザード7 レジデント イービル,672,"3,016",+100,45,4,1/61.6,1/67.0,1/754.0,2023-04-15
89040,モンキーターンⅣ,673,"2,289","-2,400",4,3,1/327.0,1/572.2,1/763.0,2023-04-15


In [7]:
a_list = a_list[:14]
a_list

['2024-08-28',
 '2024-08-26',
 '2024-08-18',
 '2024-08-16',
 '2024-08-08',
 '2024-08-06',
 '2024-07-28',
 '2024-07-26',
 '2024-07-18',
 '2024-07-16',
 '2024-07-08',
 '2024-07-06',
 '2024-06-28',
 '2024-06-26']

In [8]:
get_slot_data('2024-08-28','ロイヤル網走店')

In [58]:
from threading import Thread


# ths = [None] * len(a_list[:14])
# for i in range(len(a_list[:14])):
#     ths[i] = Thread(target=get_slot_data, args=(a_list[i],'ロイヤル網走店' ), daemon=True)
    # df = ths[i].start()
    # ths[i].join()



for d in a_list[:14]:
    try :
        df = get_slot_data(date = d,store = 'ロイヤル網走店' )
        #每個df concat起來
        if d == a_list[0]:
            df_all = df
        else:
            df_all = pd.concat([df_all, df], axis=0)
    except Exception as e:
        print(e)
        print(f'{d} is not available')

#25秒        

In [10]:

#------------------------------------------------
#用差枚總數排行
df_all['差枚'] = df_all['差枚'].fillna(0)
df_all['差枚'] = df_all['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
top10_coins = df_all.groupby('台番号')['差枚'].sum().sort_values(ascending=False).head(10)
df_all['win'] = df_all['差枚'].apply(lambda x: 1 if x > 0 else 0)
#top10.index 篩選df_all['台號'] == top10.index
df_top10_coins = df_all[df_all['台番号'].isin(top10_coins.index)]
#計算top10的WIN RATE
top10_coins_win_rate = df_top10_coins.groupby('台番号')['win'].mean().sort_values(ascending=False).head(10)
df_final_coins = pd.merge(top10_coins, top10_coins_win_rate, on='台番号', how='left')
print(df_final_coins)

#------------------------------------------------
#計算['win']裡的1的比例 groupby 台號
win_rate_top10  = df_all.groupby('台番号')['win'].mean().sort_values(ascending=False).head(10)
#merge對應的差枚
top10_win_rate_coins = df_all[df_all['台番号'].isin(win_rate_top10.index)]
top10_win_rate_coins = top10_win_rate_coins.groupby('台番号')['差枚'].sum()
df_final_win_rate = pd.merge( win_rate_top10,top10_win_rate_coins , on='台番号', how='left')
print(df_final_win_rate)

        差枚       win
台番号                 
412  32465  0.428571
520  30507  0.500000
457  24802  0.785714
427  23146  0.571429
441  20617  0.500000
459  16956  0.500000
426  15138  0.714286
442  14575  0.571429
429  14351  0.428571
447  12839  0.500000
          win     差枚
台番号                 
457  0.785714  24802
426  0.714286  15138
553  0.642857   7202
421  0.642857   8139
535  0.642857   6508
481  0.571429   5409
435  0.571429   4785
442  0.571429  14575
499  0.571429   4850
427  0.571429  23146


C:\Users\文傑\AppData\Local\Temp\ipykernel_3988\507179861.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_all['差枚'] = df_all['差枚'].str.replace(',', '').str.replace('+', '').astype(int)


以下分析

In [ ]:
df_all['差枚'] = df_all['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
#差枚欄位改為int
df_all['差枚'].astype(int)

In [ ]:
#小丑
dic = {1 : 1/163.8 ,
2 : 1/159.1
,3 : 1/148.6
,4 : 1/135.2
,5 : 1/126.8
,6 : 1/114.6}

In [ ]:
dic.values()

In [ ]:
def safe_division(x):
    try:
        return eval(x) if '/' in x else float(x)
    except ZeroDivisionError:
        return 0  # 或者其他你希望在分母為零時返回的值

df_all['合成確率'] = df_all['合成確率'].apply(safe_division)
df_all['合成確率'].astype(float)

In [ ]:
min(dic.values(), key=lambda x:abs(x-0.006))

In [ ]:
#最接近dic 的value給予其key 例如最接近 1/163.8 的值為1 , 1/159.1 的值為2
def find_nearest(dic , value):
    nearest = min(dic.values(), key=lambda x:abs(x-value))
    for k, v in dic.items():
        if v == nearest:
            return k


In [ ]:
#新增一個欄位並套用find_nearest function
df_all['合成確率_rank'] = df_all['合成確率'].apply(lambda x : find_nearest(dic , x))

In [ ]:
マイジャグラーV = df_all[df_all['機種名']=='マイジャグラーV'].sort_index(ascending=True)

In [ ]:
マイジャグラーV.to_csv('マイジャグラーV.csv', index=False)

In [ ]:
df_all.to_csv('slot_data_マルハン石和店_20240514.csv', index=False)

In [ ]:
#test for the data image

respinse = requests.get('https://ana-slo.com/2024-05-07-%e3%83%9e%e3%83%ab%e3%83%8f%e3%83%b3%e7%9f%b3%e5%92%8c%e5%ba%97-data/')
soup = BeautifulSoup(respinse.text, 'html.parser')

div = soup.find_all('div', id='tab02_0')
#找到div裡的script  
script = div[0].find_all('script')
len(script)
#提取script裡的 x_value , y_value
x_value = script[0].text.split('x_value = ')[1].split(';')[0]
y_value = script[0].text.split('y_value = ')[1].split(';')[0]
#用x_value , y_value 劃出折線圖
import matplotlib.pyplot as plt
x = json.loads(x_value)
y = json.loads(y_value)
plt.plot(x, y)
plt.show()

